In [218]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
from shapely.geometry import Polygon
import sys

In [191]:
cities = ['Dhaka Metropolitan','Ghent','Philadelphia','Tel Aviv','Washington DC']
thresholds = [300, 600, 1000]

In [193]:
warnings.filterwarnings('ignore')
cities_scores = list()
for i in cities:
    city_score = gpd.read_file(r'D:\Dumps\Scores output WP2-OSM\100m grids\Grid_geoms\gridscore_grav3_'+i+'.gpkg')
    scores_df = city_score.iloc[range(0,len(city_score),3)]['geometry'].reset_index(drop = True)
    for j in thresholds:
        ls = list()
        score = city_score['tr_'+str(j)].dropna().reset_index(drop =True)
        for k in score:
            if k == 0: 
                ls.append('0 no')
            elif k < j: # one perfect score (no route cost to UGS); score is threshold - route (and entry) cost
                ls.append('1 low')
            elif k < j*2: # two perfect scores
                ls.append('2 mediocre')
            elif k < j*3: # three perfect scores
                ls.append('3 sufficient')
            elif k < j*4: # four perfect scores
                ls.append('4 good')
            else: # above four perfect scores (or 8 UGS on half the threshold distance)
                ls.append('5 excellent')
        print(i, j)
        scores_df = pd.concat([scores_df, 
                               pd.Series(round(score, 1), name = 'Grav-Sc '+str(j)), 
                               pd.Series(ls, name = 'GravQnt '+str(j))], 
                              axis = 1)
    cities_scores.append(scores_df)

Dhaka Metropolitan 300
Dhaka Metropolitan 600
Dhaka Metropolitan 1000
Ghent 300
Ghent 600
Ghent 1000
Philadelphia 300
Philadelphia 600
Philadelphia 1000
Tel Aviv 300
Tel Aviv 600
Tel Aviv 1000
Washington DC 300
Washington DC 600
Washington DC 1000


In [194]:
cities_scores[3]

,geometry,Grav-Sc 300,GravQnt 300,Grav-Sc 600,GravQnt 600,Grav-Sc 1000,GravQnt 1000
0,"POLYGON ((34.79125 32.14625, 34.79208 32.14625...",0.0,0 no,0.0,0 no,0.0,0 no
1,"POLYGON ((34.79208 32.14625, 34.79292 32.14625...",0.0,0 no,0.0,0 no,0.0,0 no
2,"POLYGON ((34.79292 32.14625, 34.79375 32.14625...",0.0,0 no,0.0,0 no,0.0,0 no
3,"POLYGON ((34.79375 32.14625, 34.79458 32.14625...",0.0,0 no,0.0,0 no,0.0,0 no
4,"POLYGON ((34.79458 32.14625, 34.79542 32.14625...",0.0,0 no,0.0,0 no,0.0,0 no
...,...,...,...,...,...,...,...
7360,"POLYGON ((34.80125 32.06458, 34.80208 32.06458...",373.2,2 mediocre,1594.7,3 sufficient,4353.0,5 excellent
7361,"POLYGON ((34.80208 32.06458, 34.80292 32.06458...",629.8,3 sufficient,1832.2,4 good,5217.1,5 excellent
7362,"POLYGON ((34.80292 32.06458, 34.80375 32.06458...",904.6,4 good,2263.5,4 good,5960.8,5 excellent
7363,"POLYGON ((34.80375 32.06458, 34.80458 32.06458...",6900.0,5 excellent,13800.0,5 excellent,23000.0,5 excellent


In [42]:
import os
files = [pd.read_csv('D:/Dumps/E2SFCA/Scores/'+i+'.csv') for i in cities]

In [195]:
E2SCFA_class = list()
for i in cities:
    city_score = pd.read_csv('D:/Dumps/E2SFCA/Scores/'+i+'.csv')
    score_df = city_score['population']
    for j in thresholds:
        ls = list()
        score = city_score['Sc-norm '+str(j)]
        for k in score:
            if k == 0: # m2 of UGS per person and weighted with Gaussian (threshold capped) distance decay
                ls.append('0 no')
            elif k < 9: # m2 per person required by the WHO
                ls.append('1 low')
            elif k < 18: # m2 per person twice required by the WHO
                ls.append('2 mediocre')
            elif k < 30: # m2 per person preferred by the WHO
                ls.append('3 sufficient')
            elif k < 100: # over 100m2 per person, the additive value of m2 per person becomes almost obsolete
                ls.append('4 good') # below the 100m2 is still a good score
            else:
                ls.append('5 excellent') # above it will be excellent
        print(i, j)
        score_df = pd.concat([score_df, 
                              round(pd.Series(score, name = 'E2SCFA-Sc '+str(j)),2), 
                              pd.Series(ls, name = 'E2SCFA-class '+str(j))], 
                             axis = 1)
    E2SCFA_class.append(score_df)

Dhaka Metropolitan 300
Dhaka Metropolitan 600
Dhaka Metropolitan 1000
Ghent 300
Ghent 600
Ghent 1000
Philadelphia 300
Philadelphia 600
Philadelphia 1000
Tel Aviv 300
Tel Aviv 600
Tel Aviv 1000
Washington DC 300
Washington DC 600
Washington DC 1000


In [196]:
E2SCFA_class[0]

,population,E2SCFA-Sc 300,E2SCFA-class 300,E2SCFA-Sc 600,E2SCFA-class 600,E2SCFA-Sc 1000,E2SCFA-class 1000
0,39,0.0,0 no,0.00,0 no,0.00,0 no
1,37,0.0,0 no,0.00,0 no,0.00,0 no
2,33,0.0,0 no,0.00,0 no,0.00,0 no
3,41,0.0,0 no,0.00,0 no,0.00,0 no
4,41,0.0,0 no,0.00,0 no,0.00,0 no
...,...,...,...,...,...,...,...
31312,8,0.0,0 no,0.00,0 no,21.22,3 sufficient
31313,8,0.0,0 no,0.00,0 no,19.25,3 sufficient
31314,6,0.0,0 no,0.00,0 no,28.34,3 sufficient
31315,6,0.0,0 no,0.00,0 no,22.15,3 sufficient


In [202]:
scores = list()
for i in enumerate(cities):
    score = pd.concat([E2SCFA_class[i[0]], cities_scores[i[0]]], axis = 1)
    
    score = score[['geometry','population',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000',
                   'E2SCFA-Sc 300','E2SCFA-Sc 600','E2SCFA-Sc 1000',
                   'Grav-Sc 300','Grav-Sc 600','Grav-Sc 1000']]
        
    # Adding categories without grid geoms attached to ensure that each category is in a plotted map
    ls2 = pd.DataFrame(list([
                [Polygon(),'5 excellent','5 excellent','5 excellent','5 excellent','5 excellent','5 excellent'],
                [Polygon(),'4 good','4 good','4 good','4 good','4 good','4 good'],
                [Polygon(),'3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient','3 sufficient'],
                [Polygon(),'2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre','2 mediocre'],
                [Polygon(),'1 low','1 low','1 low','1 low','1 low','1 low'],
                [Polygon(),'0 no','0 no','0 no','0 no','0 no','0 no']]))
    
    ls2.columns = ['geometry',
                   'E2SCFA-class 300','E2SCFA-class 600','E2SCFA-class 1000',
                   'GravQnt 300','GravQnt 600','GravQnt 1000']
    
    score = pd.concat([score, pd.DataFrame(ls2)])
    
    score = gpd.GeoDataFrame(score, geometry = 'geometry', crs = 4326)
    score.to_file('D:/Dumps/Scores output WP2-OSM/100m grids/'+i[1]+'.gpkg')
    scores.append(score)

In [233]:
[[print(city[1], 
        scores[city[0]].groupby('E2SCFA-class '+str(t))['E2SCFA-class '+str(t)].count()) 
  for t in thresholds] 
 for city in enumerate(cities)]
sys.stdout.flush()

Dhaka Metropolitan E2SCFA-class 300
0 no            29606
1 low            1702
2 mediocre          1
3 sufficient        1
4 good              7
5 excellent         6
Name: E2SCFA-class 300, dtype: int64
Dhaka Metropolitan E2SCFA-class 600
0 no            27305
1 low            3987
2 mediocre         10
3 sufficient        6
4 good             14
5 excellent         1
Name: E2SCFA-class 600, dtype: int64
Dhaka Metropolitan E2SCFA-class 1000
0 no            26046
1 low            5228
2 mediocre         18
3 sufficient       22
4 good              8
5 excellent         1
Name: E2SCFA-class 1000, dtype: int64
Ghent E2SCFA-class 300
0 no            21666
1 low            5137
2 mediocre        403
3 sufficient      274
4 good            688
5 excellent       800
Name: E2SCFA-class 300, dtype: int64
Ghent E2SCFA-class 600
0 no            17796
1 low            8164
2 mediocre        601
3 sufficient      483
4 good           1007
5 excellent       917
Name: E2SCFA-class 600, dtype: int64